# Excel Sheet(CSV) to SQL
A script that converts a csv file (Excel Sheet) to an SQL file that can be imported into a MySQL database.

 - Works only with .csv files which confine it to be used only on single sheet Excel Files.
 - It assumes the `filename` as `table name` and `first row cells` are taken as `column names`
 - The output filename is same as the input file but with an extension `sql`


In [7]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

# Define your input file name and output file name
fname = "Copy.csv"
output_fname = 'tmp.csv'

# Remove trailing whitespace and save the result to tmp.csv
with open(fname, 'r') as input_file, open(output_fname, 'w') as output_file:
    for line in input_file:
        cleaned_line = line.rstrip(', \n') + '\n'
        output_file.write(cleaned_line)

# Get the filename without the extension
op = fname.split('.')[0]

# Create the output SQL file name
opfile = f"{op}.sql"

# Add backticks around the filename
op = f'`{op}`'

# Get the columns from the first line of tmp.csv
with open(output_fname, 'r') as tmp_file:
   first_line = tmp_file.readline().strip()
   columns = f'`{first_line}`'

In [8]:
df = pd.read_csv('tmp.csv')

# Apply string operations only to string columns
string_columns = df.select_dtypes(include=['object']).columns
df[string_columns] = df[string_columns].apply(lambda x: x.str.strip(', \t'))

df_copy = df.copy()

for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df_copy[col] = pd.to_datetime(df[col], infer_datetime_format=True, errors='raise')
        except ValueError:
            pass

# Save the cleaned DataFrame back to output_fname
df_copy.to_csv(output_fname, index=False)

In [9]:
# Generate SQL INSERT statements using Python
with open(opfile, 'w') as sql_file:
    columns = ','.join([f'`{col}`' for col in df_copy.columns])  # Wrap column names in backticks
    sql_file.write(f'INSERT INTO {op}({columns})\nVALUES\n')  # Wrap table name in backticks

    with open(output_fname, 'r') as csv_file:
        next(csv_file)  # Skip the header line
        
        for line in csv_file:
            values = line.strip().split(',')
            sql_values = ','.join([f"'{value}'" if value != '' else 'NULL' for value in values])
            sql_file.write('    ')
            sql_line = f'({sql_values}),\n'
            sql_file.write(sql_line)


with open(opfile, 'r') as sql_file:
    lines = sql_file.readlines()

lines[-1] = lines[-1].rstrip(', \n')  # Remove trailing comma from the last line

with open(opfile, 'w') as sql_file:
    sql_file.writelines(lines)

# Add a semicolon at the end
with open(opfile, 'a') as sql_file:
    sql_file.write(';\n')

In [10]:
# Clean up temporary file
os.remove(output_fname)